# Project enda : Example E


In this example we will set up a more complex dayahead power generation prediction, in order to test a backtesting operation of the dataset. 

In [ ]:
import enda
import datetime
import os
import pandas as pd
import time

from enda.contracts import Contracts
from enda.scoring import Scoring
from enda.backtesting import BackTesting

from enda.feature_engineering.datetime_features import DatetimeFeature
from enda.power_stations import PowerStations
from enda.timeseries import TimeSeries

pd.options.display.max_columns = None
pd.options.display.max_colwidth = 30

import matplotlib.pyplot as plt
#%matplotlib notebook 

In [ ]:
DIR = '.'
generation_source = ["wind", "solar", "river"]

# 1. Read and prepare data

In [ ]:
def get_example_e_dataset(source):

    if source not in ["wind", "solar", "river"]: 
            raise NotImplementedError("unknown source argument")
        
    # get station portfolio
    stations = Contracts.read_contracts_from_file(os.path.join(
        DIR, source, "stations_" + source + ".csv")
    )
    
    # display it as a multiindex with day as second index 
    stations = PowerStations.get_stations_daily(
        stations, 
        station_col='station',
        date_start_col="date_start",
        date_end_exclusive_col="date_end_exclusive"
    )
    
    # between dates of interest
    stations = PowerStations.get_stations_between_dates(
        stations, 
        start_datetime=pd.to_datetime('2017-01-01'),
        end_datetime_exclusive=pd.to_datetime('2022-01-01')
    )

    # on a 30-minutes scale
    stations = TimeSeries.interpolate_daily_to_sub_daily_data(
        stations,
        freq='30min', 
        tz='Europe/Paris', 
        index_name='time'
    )

    # integrate outages
    outages = PowerStations.read_outages_from_file(
        os.path.join(DIR, "events.csv"), 
        station_col='station',
        time_start_col="time_start", 
        time_end_exclusive_col="time_end", 
        pct_outages_col="impact_production_pct_kw",
        tzinfo="Europe/Paris"
    )    
    
    stations = PowerStations.integrate_outages(
        df_stations=stations,   
        df_outages=outages, 
        station_col='station',
        time_start_col="time_start", 
        time_end_exclusive_col="time_end", 
        installed_capacity_col="installed_capacity_kw",
        pct_outages_col="impact_production_pct_kw"
    )
    
    # get production
    production = pd.read_csv(
        os.path.join(DIR, source, "production_" + source + ".csv"), 
        parse_dates=["time"],
        date_parser=lambda col: pd.to_datetime(col, utc=True)
    )
    production['time'] = TimeSeries.align_timezone(production['time'], tzinfo='Europe/Paris')
    production.set_index(["station", "time"], inplace=True)

    production = TimeSeries.average_to_upper_freq(
        production,
        freq='30min', 
        tz='Europe/Paris',
        index_name='time', 
        enforce_single_freq=False
    )

    dataset = pd.merge(stations, production, how='inner', left_index=True, right_index=True)
    dataset = dataset.dropna()

    # get weather for wind and solar
    if source in ["wind", "solar"]:
        weather = pd.read_csv(
            os.path.join(DIR, source, "weather_forecast_" + source + ".csv"),
            parse_dates=["time"],
            date_parser=lambda col: pd.to_datetime(col, utc=True)  
        )
        weather['time'] = TimeSeries.align_timezone(weather['time'], tzinfo='Europe/Paris')
        weather.set_index(["station", "time"], inplace=True)

        weather = TimeSeries.interpolate_freq_to_sub_freq_data(
            weather,
            freq='30min', 
            tz='Europe/Paris',
            index_name='time',
            method="linear"
        )
    
        dataset = pd.merge(dataset, weather, how='inner', left_index=True, right_index=True)
  
    # featurize for solar
    if source == "solar":
        dataset = DatetimeFeature.split_datetime(
            dataset, split_list=['minuteofday', 'dayofyear']
        )
        
        dataset = DatetimeFeature.encode_cyclic_datetime_index(
            dataset, split_list=['minuteofday', 'dayofyear']
        )

    return dataset

In [ ]:
%%time 
dataset_wind = get_example_e_dataset("wind")

In [ ]:
%%time 
dataset_solar = get_example_e_dataset("solar")

In [ ]:
%%time 
dataset_river = get_example_e_dataset("river")

In [ ]:
dataset = dict(zip(generation_source, [dataset_wind, dataset_solar, dataset_river]))

In [ ]:
# Compute load factor
# We drop the power_kw information during that step, not to bias the IA algorithm afterwards. 
def wrapper_compute_load_factor(df):
    return enda.PowerStations.compute_load_factor(
               df, 
               installed_capacity_kw='installed_capacity_kw', 
               power_kw='power_kw',
               drop_power_kw=True
           )           

dataset_final = {source: wrapper_compute_load_factor(d) for source, d in dataset.items()}

# 2. Make a basic prediction

### Separe between training and forecasting dataset to backtest the data

We have here the full datasets which have been built using the Enda utilities function, and some historical information gathered from the TSO, diverse meteo information suppliers, and contracts data. 

We will now distinguish our full datasets in two, in order to obtain the training and forecasting datasets representative of what could be obtained in real life condition. 

In [ ]:
# wrapper function around the 
def separate_train_test_sets(df):
    
    # let's create the input train dataset
    train_set = df[df.index.get_level_values(1) < pd.to_datetime('2021-12-01 00:00:00+01:00')]
        
    # let's create the input data for our forecast
    forecast_set = df[df.index.get_level_values(1) >= pd.to_datetime('2021-12-01 00:00:00+01:00')]
    forecast_set = forecast_set.drop(columns="load_factor")
    
    # and let us keep the information of the real power generation for testing purposes
    future_set = df[df.index.get_level_values(1) >= pd.to_datetime('2021-12-01 00:00:00+01:00')]

    return train_set, forecast_set, future_set

train_test_future_sets = {source: separate_train_test_sets(data) for source, data in dataset_final.items()} 

train_set = {source: train_test_future_sets[source][0] for source in generation_source}
forecast_set = {source: train_test_future_sets[source][1] for source in generation_source}
future_set = {source: train_test_future_sets[source][2] for source in generation_source}


In [ ]:
train_set["wind"].shape

Let's use the enda algorithms to make a simple power prediction. 

In [ ]:
# import power predictors
from enda.power_predictor import PowerPredictor

### Run of river prediction

In [ ]:
# import a dummy ML backend for river
from enda.estimators import EndaEstimatorRecopy

# build a PowerPredictor object
river_predictor = PowerPredictor(standard_plant=False)

# use PowerPredictor to train the estimator from the run of river data, 
# and from a naive recopy estimator
river_predictor.train(train_set["river"], estimator=EndaEstimatorRecopy(period='1D'), target_col="load_factor")

In [ ]:
train_set["river"]

In [ ]:
# Once it has been trained, we can predict the power for each power plant individually, calling predict()
# from PowerPredictor()
pred_river = river_predictor.predict(forecast_set["river"], target_col="load_factor")

In [ ]:
pred_river

### Wind prediction

In [ ]:
# boot up an H20 server
import h2o
h2o.init(nthreads=-1)
h2o.no_progress()

In [ ]:
# enda's wrapper around H2O models
from enda.ml_backends.h2o_estimator import EndaH2OEstimator  
from h2o.estimators import H2OGradientBoostingEstimator
from h2o.estimators import H2OGeneralizedLinearEstimator

# define an estimator
gradboost_estimator = EndaH2OEstimator(H2OGradientBoostingEstimator(
    ntrees=500,
    max_depth=5,
    sample_rate=0.5,
    min_rows=5,
    seed=17
))

In [ ]:
# build a PowerPredictor object
wind_predictor = PowerPredictor(standard_plant=True)

In [ ]:
# train the estimator
wind_predictor.train(train_set["wind"], estimator=gradboost_estimator, target_col="load_factor")

In [ ]:
# predict
pred_wind = wind_predictor.predict(forecast_set['wind'], target_col="load_factor", is_positive = True)

In [ ]:
pred_wind

### Solar prediction

In [ ]:
# keep the best estimator from h2o

# build a PowerPredictor object
solar_predictor = PowerPredictor(standard_plant=True)

# use the same good estimator
gradboost_estimator = EndaH2OEstimator(H2OGradientBoostingEstimator(
    ntrees=500,
    max_depth=5,
    sample_rate=0.5,
    min_rows=5,
    seed=17
))

# train the estimator
solar_predictor.train(train_set["solar"], estimator=gradboost_estimator, target_col="load_factor")

# predict
pred_solar= solar_predictor.predict(forecast_set["solar"], target_col="load_factor", is_positive=True)

In [ ]:
pred_solar

In [ ]:
# shutdown your h2o local server
h2o.cluster().shutdown()
# wait for h2o to really finish shutting down
time.sleep(5)

### Getting back to power prediction

To get back to power prediction, we simply need to use the installed capacity field and multiply it by the load factor to find again the power (kw)

In [ ]:
# we start by merging again the installed_capacity (kw) field

def merge_stations_and_features(df1, df2):
    df = pd.merge(df1, df2, how='inner', left_index=True, right_index=True)
    return df.dropna()

pred = dict(zip(generation_source, [pred_wind, pred_solar, pred_river]))
prediction = {source: merge_stations_and_features(
                          forecast_set[source].loc[:, ["installed_capacity_kw"]], 
                          pred[source])
              for source in generation_source
             }

In [ ]:
# We drop the load_factor information during that step.
def wrapper_compute_power_kw_from_load_factor(df):
    return enda.PowerStations.compute_power_kw_from_load_factor(
               df, 
               installed_capacity_kw='installed_capacity_kw', 
               load_factor='load_factor',
               drop_load_factor=True
           )           

prediction = {source: wrapper_compute_power_kw_from_load_factor(p) 
             for source, p in prediction.items()}

In [ ]:
prediction["river"]

# 3. Plots and KPI

### Plot predicted data along with the real production 

In [ ]:
# Get back to the power_kw
real = {source: wrapper_compute_power_kw_from_load_factor(r) 
             for source, r in future_set.items()}


fig, axis = plt.subplots(3, 1, figsize=(9, 12), sharex=True, sharey=False)

i = 0
for source, data in prediction.items():
    axis[i].grid(True)
    axis[i].plot(data["power_kw"].groupby(level=1).agg("sum"), label="prediction", c="blue")
    axis[i].set_xlabel('Date', fontsize=12)
    axis[i].set_ylabel('Production (kW)', fontsize=12)
    axis[i].set_title(source)
    i+=1
    
i = 0
for source, data in real.items():
    axis[i].plot(data["power_kw"].groupby(level=1).agg("sum"), label="real", c="red")
    axis[i].set_xlabel('Date', fontsize=12)
    axis[i].set_ylabel('Production (kW)', fontsize=12)
    axis[i].legend()
    i +=1

fig.tight_layout()

### Compute the nMAPE

In [ ]:
# create the benchmark dataframe for wind power plant

# we keep the active capacity, the actual power injection, and the power prediction
def build_becnhmark(source):
    benchmark = pd.merge(real[source][["installed_capacity_kw", "power_kw"]], 
                         prediction[source]["power_kw"].to_frame(), 
                         how="inner", left_index=True, right_index=True)
    benchmark = benchmark.rename({"power_kw_x": "actual", 
                                   "power_kw_y": "enda",
                                 },
                                 axis=1)
    return benchmark

benchmark = {source: build_becnhmark(source)for source in generation_source}

benchmark['wind']

In [ ]:
# sum over all power plants
benchmark_portfolio = {source: benchmark[source].groupby(level="time").sum() for source in generation_source}

In [ ]:
# define a scoring
scoring_benchmark = {source: Scoring(benchmark_portfolio[source], 
                                     target="actual",
                                     normalizing_col="installed_capacity_kw") for source in generation_source}

In [ ]:
# compute the nAE
nAE = {source: scoring_benchmark[source].normalized_absolute_error() for source in generation_source}
nAE['wind']

In [ ]:
nMAPE = {source: nAE[source].mean() for source in generation_source}
nMAPE

It is a 7% difference (not exactly a percentage) for wind, 2% for solar, and 14% for run of river. 

# 4. Perform a benchmark with backtesting

As in example B_load, we will perform a backtesting of the data we gathered, week after week. With the given dataset, this means :
- for each week w from early 2020 until the end of the dataset : train using data from the beginning of the dataset (early 2018) until a few days before week w, then eval on w. 
    - the first iteration will train an algorithm using data from 2018 to 2019, then eval on the first week of 2020
    - the second iteration will train using data from 2018 to a bit before the first week of 2020, then eval on the second week of 2020
    - and so on...
- keep the predictions of each time-step using this method, from early 2020 to december 2021.  
- then compare these predictions to the historic data to evaluate the quality of each algorithm. 

This makes most sense if in your production environment, you plan to retrain the algorithm regularly with recent data. 

We'll just perform it for wind turbines

In [ ]:
# boot up an H20 server
import h2o
h2o.init(nthreads=-1)
h2o.no_progress()

In [ ]:
# we'll test simple estimators from Sklearn we haven't tried yet, for demonstration purposes. 
from enda.ml_backends.sklearn_estimator import EndaSklearnEstimator
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, SGDRegressor

# define a dict of estimators
all_estimators= dict()


all_estimators['sklearn_lin_reg'] = EndaSklearnEstimator(LinearRegression())  

all_estimators['sklearn_sgd'] = EndaSklearnEstimator(
    Pipeline([('standard_scaler', StandardScaler()),
              ('sgd', SGDRegressor())
             ]
            )
)

all_estimators['h2o_gboost'] = EndaH2OEstimator(H2OGradientBoostingEstimator(
    ntrees=500,
    max_depth=5,
    sample_rate=0.5,
    min_rows=5,
    seed=17
))


In [ ]:
# create a PowerPredictor
predictor = PowerPredictor(standard_plant=True)

In [ ]:
# run the backtesting and fill a benchmark_wind dataframe with the results from the different algorithms

start_backtesting_dt = pd.to_datetime('2020-01-01 00:00:00+01:00').tz_convert('Europe/Paris')
benchmark_wind = dataset_final['wind'][dataset_final['wind'].index.get_level_values('time') \
                                       >= start_backtesting_dt]["load_factor"].to_frame("actual_load_factor")

days_in_each_iteration = 28

for estimator_name, estimator in all_estimators.items():

    count_iterations = 0
    
    estimator_predictions = []

    for train_set, test_set in BackTesting.yield_train_test(
        dataset_final['wind'],
        start_eval_datetime=start_backtesting_dt,
        days_between_trains=days_in_each_iteration,
        gap_days_between_train_and_eval=14
    ):
        count_iterations += 1
        if count_iterations <= 2 or count_iterations % 10 == 0:
            print("Model {}, backtesting iteration {}, train set {}->{}, test set {}->{}\n".format(
                   estimator_name, count_iterations, 
                   train_set.index.get_level_values('time').min(),
                   train_set.index.get_level_values('time').max(), 
                   test_set.index.get_level_values('time').min(), 
                   test_set.index.get_level_values('time').max()))
        
        # featurize
        test_set = test_set.drop(columns=["load_factor"])
        
        # train and predict
        predictor.train(train_set, estimator=estimator, target_col='load_factor')
        estimator_predictions.append(predictor.predict(test_set, target_col='load_factor', is_positive=True))
    
    benchmark_wind[estimator_name] = pd.concat(estimator_predictions)


In [ ]:
# don't forget to shutdown your h2o local server
h2o.cluster().shutdown()
# wait for h2o to really finish shutting down
time.sleep(5)

In [ ]:
benchmark_wind

In [ ]:
# add the installed_capacity
benchmark_wind_kw = pd.merge(benchmark_wind, dataset_final['wind']["installed_capacity_kw"],
                          how='inner', left_index=True, right_index=True)

benchmark_wind_kw = (benchmark_wind_kw.multiply(benchmark_wind_kw["installed_capacity_kw"], axis=0)
                                      .drop(columns="installed_capacity_kw")
                                      .rename({"actual_load_factor": "actual_power_kw"}, axis=1)
        )

benchmark_wind_kw = pd.merge(benchmark_wind_kw, dataset_final['wind']["installed_capacity_kw"],
                          how='inner', left_index=True, right_index=True)
benchmark_wind_kw

In [ ]:
# visualize predictions

n_estimators = len(all_estimators) # = len(benchmark_wind.columns)-1

fig, axis = plt.subplots(n_estimators , 1, figsize=(9, 4 * n_estimators), sharex=True, sharey=False)

i = 0
for c in benchmark_wind.columns:
    if c != "actual_load_factor":
        axis[i].grid(True)
        axis[i].plot(benchmark_wind_kw["actual_power_kw"].groupby(level=1).agg("sum"), label="actual", c="blue")
        axis[i].plot(benchmark_wind_kw[c].groupby(level=1).agg("sum"), label=c, c="orange")
        axis[i].set_xlabel('Date', fontsize=12)
        axis[i].set_ylabel('Production (kW)', fontsize=12)
        #axis[i].set_title(source)
        axis[i].legend()
        i+=1
        
fig.tight_layout()

In [ ]:
# sum over all power plants
benchmark_wind_portfolio = benchmark_wind_kw.groupby(level="time").sum()

In [ ]:
# define a scoring
scoring_benchmark = Scoring(benchmark_wind_portfolio , target="actual_power_kw", normalizing_col="installed_capacity_kw") 

In [ ]:
# compute the nAE
nAE = scoring_benchmark.normalized_absolute_error()
nAE

In [ ]:
nMAPE = nAE.mean()
nMAPE

# Conclusion

Do it with solar or run of river !